# [Denoising Diffusion Probabilistic Models(DDPM)](https://nn.labml.ai/diffusion/ddpm/index.html)

This is a PyTorch Implementation of the paper Denosing Diffusion Probabilistic Models.

### Forward Process
The forward process adds noise to the data $x_0 \sim q(x_0)$, for T timesteps

$$ q(x_t|x_{t-1}) = N(x_t; \sqrt{1 - \beta_t} x_{t-1}, \beta_t I)

In [ ]:
class DenoiseDiffusion:
    def __init__(self, eps_model, n_steps, device):
        super().__init__()
        self.eps_model = eps_model
        self.beta = torch.linspace(0.0001, 0.02, n_steps).to(device)
        self.alpha = 1. - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, dim=0)
        self.n_steps = n_steps
        self.sigma2 = self.beta
    
    def q_xt_x0(self, x0, t):
        mean = gather(self.alpha_bar, t) ** 0.5 * x0
        var = 1 - gather(self.alpha_bar, t)
        return mean, var
    
    def q_sample(self, x0, t, eps):
        if eps is None:
            eps = torch.randn_like(x0)
        
        mean, var = self.q_xt_x0(x0, t)
        return mean + (var ** 0.5) * eps
    
    def p_sample(self, xt, t):
        eps_theta = self.eps_model(xt, t)
        alpha_bar = gather(self.alpha_bar, t)
        alpha = gather(self.alpha, t)
        eps_coef = (1 - alpha) / (1 - alpha_bar) ** .5
        mean = 1 / (alpha)
        